In [1]:
import tensorflow
import numpy
import transformers
import datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 2)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [3]:
dataset = load_dataset("nsmc")

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train = dataset['train']
test = dataset['test']

In [5]:
test

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 50000
})

In [6]:
import pandas as pd
from datasets import Dataset

# train 데이터를 Dataset 형식으로 불러온 경우
train_df = train.to_pandas()  # Dataset을 DataFrame으로 변환

# NaN 값 제거
train_cleaned = train_df.dropna()
train_cleaned = train_cleaned.drop_duplicates(subset=['document'])
train_cleaned

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [7]:
test_df = test.to_pandas()  # Dataset을 DataFrame으로 변환

# NaN 값 제거
test_cleaned = test_df.dropna()
test_cleaned = test_cleaned.drop_duplicates(subset=['document'])
test_cleaned

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [8]:
min_length_sen = 10
max_length_sen = 20

filtered_df = train_cleaned[(train_cleaned['document'].str.len() > min_length_sen) &
                            (train_cleaned['document'].str.len() < max_length_sen)]

test_filtered_df = test_cleaned[(test_cleaned['document'].str.len() > min_length_sen + 5) &
                            (test_cleaned['document'].str.len() < max_length_sen - 3)]



filtered_df
test_filtered_df

,id,document,label
10,7462111,괜찮네요오랜만포켓몬스터잼밌어요,1
15,10268521,소위 ㅈ문가라는 평점은 뭐냐?,1
91,9992247,후속작 계획은 없나요..? ㅜ,1
96,9665771,완전 재밌엇는데 왜 평점이??,1
137,3830959,재밌었다.진저 너무 무서버~ㅠ,1
...,...,...,...
49695,2656539,후.. 인내심을 길러주는 영화,0
49740,9243476,잔잔하게 흐르는 강물같은 영화,1
49790,2221441,최악 최악 !! 어이가 없었다,0
49852,8732499,정말..유아용이였습니다....,0


In [9]:
# dataframe 데이터를 dict 내부에 list로 변경
train_dataset = filtered_df.to_dict('list')
# val_dataset = val_df.to_dict('list')
test_dataset = test_filtered_df.to_dict('list')

# # Huggingface dataset
tf_train_dataset = Dataset.from_dict(train_dataset)
# tf_val_dataset = Dataset.from_dict(val_dataset)
tf_test_dataset = Dataset.from_dict(test_dataset)

In [10]:
tf_test_dataset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 1345
})

In [11]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

hf_train_dataset = tf_train_dataset.map(transform, batched=True)
# hf_eval_dataset = tf_val_dataset.map(transform, batched=True)
hf_test_dataset = tf_test_dataset.map(transform, batched=True)

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [12]:
hf_train_dataset = hf_train_dataset.rename_column("label", "labels")
hf_train_dataset.set_format("torch")

hf_test_dataset = hf_test_dataset.rename_column("label", "labels")
hf_test_dataset.set_format("torch")

# train_dataset = tokenized_datasets["train"]
# test_dataset = tokenized_datasets["test"]

In [13]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments_dbs = TrainingArguments(
    output_dir,                       
    evaluation_strategy="epoch",         
    learning_rate = 2e-5,        # 2e-5   
    per_device_train_batch_size = 16,
#     per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    group_by_length=False,
)

In [14]:
for name, param in huggingface_model.named_parameters():
    if "embeddings" in name: 
        param.requires_grad = False
    if "encoder.layer" in name and int(name.split(".")[3]) < 11:

        param.requires_grad = False

In [15]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [16]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments_dbs,           # TrainingArguments을 통해 설정한 arguments
    train_dataset= hf_train_dataset,
#     eval_dataset = hf_eval_dataset, # training dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 35529
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6663


Step,Training Loss
500,0.604800
1000,0.534900
1500,0.552300
2000,0.479900
2500,0.471900
3000,0.484600
3500,0.483000
4000,0.447500
4500,0.447200
5000,0.423900


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoi

TrainOutput(global_step=6663, training_loss=0.4704070213623596, metrics={'train_runtime': 4692.3955, 'train_samples_per_second': 22.715, 'train_steps_per_second': 1.42, 'total_flos': 2.804421805765632e+16, 'train_loss': 0.4704070213623596, 'epoch': 3.0})

In [17]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 1345
  Batch size = 8


{'eval_loss': 0.37379691004753113,
 'eval_accuracy': 0.8364312267657993,
 'eval_f1': 0.8478561549100968,
 'eval_runtime': 51.2808,
 'eval_samples_per_second': 26.228,
 'eval_steps_per_second': 3.296,
 'epoch': 3.0}

In [18]:
del model

NameError: name 'model' is not defined

In [ ]:
del huggingface_model

In [ ]:
import torch

# GPU 사용 가능 여부 및 GPU 정보 출력
if torch.cuda.is_available():
    print("GPU 사용 가능")
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("GPU 사용 메모리: {:.2f} MB".format(torch.cuda.memory_allocated(0) / 1024 ** 2))
    print("GPU 여유 메모리: {:.2f} MB".format(torch.cuda.memory_reserved(0) / 1024 ** 2))
else:
    print("GPU 사용 불가능")